In [2]:
# !unzip -uq "dataset_imageprocessing.zip" -d "dataset_imageprocessing"

In [1]:
from keras.models import Sequential
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers import Reshape, Permute, Activation, Input, merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

import sys
import os
import glob
import cv2, numpy as np

Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dt

In [2]:
import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sessions = tf. Session(config=config)

In [3]:
# Transfer learning(MobileNetV2)

from keras.models import Sequential, Model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.applications.mobilenet_v2 import MobileNetV2  
from keras.applications.mobilenet import MobileNet

input_shape = (400, 400, 3)
# base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

# 미세 조정
base_model.trainable=True

set_trainable=False
for layer in base_model.layers:
    if layer.name == 'out_relu': # 쌓은 층 직전까지 동결
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


x = Flatten()(base_model.output)
# x = Dense(256, activation='relu')(x)
dropout = Dropout(0.6)(x)
output = Dense(3, activation='softmax', name='predictions')(dropout)
model = Model(inputs=base_model.input, output=output)

model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])

model.summary()

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/keras_applications/mobilenet_v2.py:294: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '


Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 400, 400, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 401, 401, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 200, 200, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 200, 200, 32) 128         Conv1[0][0]                      
____________________________________________________________________________________________

/home/ubuntu/anaconda3/envs/amazonei_tensorflow_p36/lib/python3.6/site-packages/ipykernel_launcher.py:29: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("pr...)`


In [4]:
# CNN model
# cnn2

from keras.layers import Dense, Input, Dropout, GlobalAveragePooling2D, Flatten, Conv2D, BatchNormalization, Activation, MaxPooling2D
from keras.models import Model, Sequential
from keras.optimizers import Adam

# number of possible label values
nb_classes = 3

# Initialising the CNN
model = Sequential()

# 1 - Convolution
model.add(Conv2D(64,(3,3), padding='same', input_shape=(400, 400, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 2nd Convolution layer
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 3rd Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# 4th Convolution layer
model.add(Conv2D(512,(3,3), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flattening
model.add(Flatten())

# Fully connected layer 1st layer
model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected layer 2nd layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(nb_classes, activation='softmax'))

opt = Adam(lr=0.0001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 400, 400, 64)      1792      
_________________________________________________________________
batch_normalization_1 (Batch (None, 400, 400, 64)      256       
_________________________________________________________________
activation_1 (Activation)    (None, 400, 400, 64)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 200, 200, 64)      0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 200, 200, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 200, 200, 128)     204928    
_________________________________________________________________
batch_normalization_2 (Batch (None, 200, 200, 128)    

In [4]:
# ImagedataGenerator 불러오기
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# train data load 
train_set = train_datagen.flow_from_directory('dataset_imageprocessing/dataset_imageprocessing/train/', 
                                              target_size=(400, 400), # 400, 400 
                                              class_mode='categorical', 
                                              batch_size=16)

# val data load 
test_set = test_datagen.flow_from_directory('dataset_imageprocessing/dataset_imageprocessing/test/', 
                                            target_size=(400, 400), # 400, 400
                                            class_mode='categorical', 
                                            batch_size=16) # suffle 적용/적용하지 않기

# fit model
history = model.fit_generator(train_set,
                              epochs = 50,
                              steps_per_epoch=128,
                             validation_data = test_set)

Found 35144 images belonging to 3 classes.
Found 8785 images belonging to 3 classes.

Epoch 1/50
128/128 [==============================] - 61s 478ms/step - loss: 5.9366 - accuracy: 0.6934 - val_loss: 28.0733 - val_accuracy: 0.4891
Epoch 2/50
128/128 [==============================] - 56s 438ms/step - loss: 4.9670 - accuracy: 0.7832 - val_loss: 0.0000e+00 - val_accuracy: 0.4813
Epoch 3/50
128/128 [==============================] - 57s 444ms/step - loss: 4.5003 - accuracy: 0.8179 - val_loss: 38.3309 - val_accuracy: 0.4557
Epoch 4/50
128/128 [==============================] - 57s 445ms/step - loss: 4.7413 - accuracy: 0.8389 - val_loss: 5.6260 - val_accuracy: 0.5713
Epoch 5/50
128/128 [==============================] - 57s 442ms/step - loss: 4.8218 - accuracy: 0.8545 - val_loss: 0.0000e+00 - val_accuracy: 0.5614
Epoch 6/50
128/128 [==============================] - 56s 441ms/step - loss: 5.1666 - accuracy: 0.8555 - val_loss: 0.0000e+00 - val_accuracy: 0.5588
Epoch 7/50
128/128 [==========

In [8]:
print(test_set.class_indices)

{'E01': 0, 'E02': 1, 'E03': 2}


In [5]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('loss, acc')
plt.xlabel('epoch')
plt.ylabel('loss, acc')
plt.legend(['train loss', 'test loss', 'train acc', 'test acc'])
plt.show()

<Figure size 640x480 with 1 Axes>

In [6]:
pred_datagen = ImageDataGenerator(rescale=1./255)

pred_set = pred_datagen.flow_from_directory('dataset_pred/', 
                                              target_size=(400, 400),
                                              class_mode='categorical',
                                              shuffle = False)

Found 8640 images belonging to 3 classes.


In [29]:
pred_set1[0]

(array([[[[0.76470596, 0.8117648 , 0.80392164],
          [0.76470596, 0.8117648 , 0.80392164],
          [0.76470596, 0.8117648 , 0.80392164],
          ...,
          [0.64705884, 0.68235296, 0.6784314 ],
          [0.64705884, 0.68235296, 0.6784314 ],
          [0.64705884, 0.68235296, 0.6784314 ]],
 
         [[0.76470596, 0.8117648 , 0.80392164],
          [0.76470596, 0.8117648 , 0.80392164],
          [0.76470596, 0.8117648 , 0.80392164],
          ...,
          [0.64705884, 0.68235296, 0.6784314 ],
          [0.64705884, 0.68235296, 0.6784314 ],
          [0.64705884, 0.68235296, 0.6784314 ]],
 
         [[0.76470596, 0.8117648 , 0.80392164],
          [0.76470596, 0.8117648 , 0.80392164],
          [0.76470596, 0.8117648 , 0.80392164],
          ...,
          [0.64705884, 0.68235296, 0.6784314 ],
          [0.64705884, 0.68235296, 0.6784314 ],
          [0.64705884, 0.68235296, 0.6784314 ]],
 
         ...,
 
         [[0.18039216, 0.19215688, 0.21176472],
          [0.18823

In [59]:
pred.shape

(8640, 3)

In [61]:
for i in range(8640):
    print(np.argmax(pred[i]), end='')

2000010011000111100101101220111011010110001001100121102011110010001000111201100001002100110212010010110001200001011111110001111100011100000001101010001110210101210111210001010010100100211001011111122111101100110111101110000002111102110101111101011001100110011111012010111011010010000011121110110110011001000101101101100100100011001010101120010110100101220000001011121001121011210100111111011201011111110110011100011011000100011100111111201000100001010010010112110101210110001101101112111100100111101001001010112112000000010001010210110011010000001011121010010010111100100100001100010101111101011202001110010000000110010111001100002111101100011012001101011020101001001110110000001011010101000012001111100211001011000100000100112010011100000010100002010101110100001100011000011010021110101100100001111111011001010010002112010011011010000000001011100011001221011001010112011210101111110001000100000100012101011001010110000110002101110101011111011100110010110111111111110011010101000001110100101102010010

In [7]:
# model 검증
from sklearn.metrics import classification_report

pred = model.predict_generator(pred_set)

val_preds = np.argmax(pred, axis=-1)
val_trues = pred_set.classes

print(classification_report(val_trues,val_preds))

              precision    recall  f1-score   support

           0       0.59      0.57      0.58      2880
           1       0.55      0.66      0.60      2880
           2       0.71      0.58      0.64      2880

   micro avg       0.61      0.61      0.61      8640
   macro avg       0.62      0.61      0.61      8640
weighted avg       0.62      0.61      0.61      8640



In [ ]:
pred = model.predict_generator(pred_set)


val_preds = np.argmax(pred, axis=-1)
val_trues = pred_set.classes
cm = metrics.confusion_matrix(val_trues, val_preds)


labels = pred_set1.class_indices.keys()
precisions, recall, f1_score, _ = metrics.precision_recall_fscore_support(val_trues, val_preds, labels=labels)

In [9]:
# 무표정
co=0
for i in range(2880):
    result = np.argmax(pred[i])
#     print(result, end='')
    if result==0:
        co+=1
        
print('\n','Neutral accuracy :', "%.2f" % (co/2880.))

# 기쁨
co=0
for i in range(2880, 5760):
    result = np.argmax(pred[i])
#     print(result, end='')
    if result==1:
        co+=1
        
print('\n','Positive accuracy :', "%.2f" % (co/2880.))


# 찡그림
co=0
for i in range(5760,8640):
    result = np.argmax(pred[i])
#     print(result, end='')
    if result==2:
        co+=1
        
print('\n','Negative accuracy :', "%.2f" % (co/2880.))

# transfer learning(mobileNetV2)
#  Neutral accuracy : 0.45
#  Positive accuracy : 0.69
#  Negative accuracy : 0.20

# transfer learning(mobileNetV2)
#  Neutral accuracy : 0.21
#  Positive accuracy : 0.82
#  Negative accuracy : 0.65

# CNN model
#  Neutral accuracy : 0.99
#  Positive accuracy : 0.79
#  Negative accuracy : 0.98

NameError: name 'pred' is not defined

In [8]:
model.save('./save/cnn0520(imagegenerator, 400).h5')
print('모델 저장이 완료되었습니다.')

모델 저장이 완료되었습니다.


In [8]:
model.save('./save/transferlearning0520(imagegenerator, 400).h5')
print('모델 저장이 완료되었습니다.')

모델 저장이 완료되었습니다.
